### Импорт библиотек

In [2]:
import pandas as pd
import numpy as np
import pickle

### Загрузка файлов

In [3]:
# Загрузка моделей
with open('models/catboost_model.pkl', 'rb') as f:
    catboost_model = pickle.load(f)

with open('models/xgboost_model.pkl', 'rb') as f:
    xgboost_model = pickle.load(f)

with open('models/randomforest_model.pkl', 'rb') as f:
    randomforest_model = pickle.load(f)

# Загрузка тестовых данных
test_data = pd.read_excel('data/test_prepared.xlsx')

### Прогнозирование

In [4]:
# # Подготовка данных для теста
# X_test = test_data.drop(columns=["dt", "Цена на арматуру"])
# y_test = test_data["Цена на арматуру"]

# # Получение предсказаний для всех моделей
# y_pred_catboost = catboost_model.predict(X_test)
# y_pred_xgboost = xgboost_model.predict(X_test)
# y_pred_rf = randomforest_model.predict(X_test)

# # Округление предсказанных значений и сохранение в массив
# y_pred_catboost = np.round(y_pred_catboost).astype(int)
# y_pred_xgboost = np.round(y_pred_xgboost).astype(int)
# y_pred_rf = np.round(y_pred_rf).astype(int)

# # Массив предсказанных значений
# predictions = [y_pred_catboost, y_pred_xgboost, y_pred_rf]
# model_names = ['CatBoost', 'XGBoost', 'RandomForest']

# # Функция для рекомендации объема тендера
# def main_recommend_tender_volume_(predictions, model_names, test_data):
#     def perenos_():
#         print('\n******************************************************************\n')

#     def recommend_tender_volume(predicted_price, current_price):
#         if predicted_price < current_price * 0.95:
#             return max(1, int((current_price - predicted_price) / 1000))
#         elif predicted_price > current_price * 1.05:
#             return max(6, int((predicted_price - current_price) / 1000))
#         else:
#             return 3

#     current_price = test_data['Цена на арматуру'].iloc[-1]  # цена на арматуру сегодня

#     for preds, model_name in zip(predictions, model_names):
#         recommended_volumes = []

#         # Вычисление тендера
#         for predicted_price in preds:
#             recommended_volume = recommend_tender_volume(predicted_price, current_price)
#             recommended_volumes.append(recommended_volume)

#         differences = preds - y_test.values
        
#         # Проверка предсказанных значений
#         print(f"Рекомендуемый объем тендера на арматуру для тестовых данных на основе {model_name}: {recommended_volumes}")
#         perenos_()
#         print(f"Предсказанные цены на тестовом наборе для {model_name}: {preds}\n")
#         print(f"Фактические цены на тестовом наборе: {y_test.values}\n")
#         print(f"Разницы между предсказанными и фактическими значениями: {differences}\n")

#         # Добавление рекомендаций в test_data
#         test_data[f'Объём {model_name}'] = recommended_volumes
#         print(f"Столбец объёма {model_name}, успешно добавлен!")
#         perenos_()

# # Вызов функции
# main_recommend_tender_volume_(predictions, model_names, test_data)

In [5]:
def predict_future(model, ca, N, features):
    future_predictions = []
    last_row = ca.iloc[-1][features].values.reshape(1, -1)
    
    for i in range(N):
        pred = model.predict(last_row)
        future_predictions.append(pred[0])
        
        last_row = last_row.copy()
        last_row[0, features.index('Номер_недели')] += 1  
        last_row[0, features.index('Год')] = ca['Год'].iloc[-1]  
        last_row[0, features.index('Цена_на_арматуру_lag2')] = last_row[0, features.index('Цена_на_арматуру_lag1')]
        last_row[0, features.index('Цена_на_арматуру_lag1')] = pred  

    return future_predictions

N = int(input("Введите количество недель для предсказания (от 1 до 6): "))
if N < 1 or N > 6:
    print("Введите число от 1 до 6!")
else:
    future_predictions = predict_future(model, ca, N, ['Год', 'Номер_недели', 'Цена_на_арматуру_lag1', 'Цена_на_арматуру_lag2'])
    print(f"Прогноз на {N} недель вперед: {future_predictions}")

NameError: name 'model' is not defined

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(ca['dt'], y, label='Реальные значения', color='blue')

future_dates = pd.date_range(start=ca['dt'].iloc[-1] + pd.Timedelta(weeks=1), periods=N, freq='W-MON')

plt.plot(future_dates, future_predictions, label='Предсказанные значения', color='red', linestyle='-')

plt.xlabel('Дата')
plt.ylabel('Цена на арматуру')
plt.title(f'{N} недель вперед')
plt.legend()
plt.xticks(rotation=45)
plt.show()

Сохранение результатов

In [ ]:
# Оставляем только нужные столбцы
columns_to_keep = ['dt', 'Цена на арматуру', 'Объём CatBoost', 'Объём XGBoost', 'Объём RandomForest']
test_data_filtered = test_data[columns_to_keep]

# Сохранение обновленного DataFrame в test.xlsx
test_data_filtered.to_excel('data/test_done.xlsx', index=False)

### Выводы

1. CatBoost: Модель показывает хорошую производительность, но имеются значительные расхождения в предсказаниях, особенно в случае высоких разниц.

2. XGBoost: Модель работает достаточно хорошо, с меньшими расхождениями, чем CatBoost, что указывает на её эффективность.

3. RandomForest: Модель также показывает хорошие результаты, но наблюдаются большие расхождения, особенно в некоторых предсказаниях.

Вывод: XGBoost лучшая модель